In [1]:
import pydicom
import os
import numpy as np
import pandas as pd
import pylidc as pl
import matplotlib.pyplot as plt
import pdb
import scipy.ndimage
import scipy.sparse
from tqdm import tqdm_notebook
import scipy
from pylidc.utils import consensus
from skimage import measure, morphology
from skimage.morphology import ball, dilation, disk
%matplotlib inline

In [2]:
LIDC_PATH = '/data/datasets/LIDC-IDRI/'
# annotations = pd.read_csv('/data/datasets/LIDC-IDRI/annotations.csv')
LIDC_IDs = os.listdir(f'{LIDC_PATH}LIDC-IDRI')
LIDC_IDs = [i for i in LIDC_IDs if 'LIDC' in i]
LIDC_IDs = np.sort(LIDC_IDs)

## Functions

In [3]:
# Load the scans in given folder path
def load_scan(all_scans):
    slices = [pydicom.dcmread(s) for s in all_scans]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices

In [4]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    outside_image = image.min() # Modified by OMM
    image[image == outside_image] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [5]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
#     pdb.set_trace()
    # modified by OMM 
    spacing = np.array([float(scan[0].SliceThickness), float(scan[0].PixelSpacing[0]), float(scan[0].PixelSpacing[1])], dtype=np.float32)
    
    resize_factor = spacing / np.asarray(new_spacing)
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

In [6]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None

def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

## Loop

In [7]:
# Use this one
path_dest = f'/data/OMM/Datasets/LIDC_other_formats/LIDC_preprocessed_3D/' 
# Get all the scans for X patient(s)
df = pd.read_csv('/data/datasets/LIDC-IDRI/annotations.csv')
scans_with_errors, scans_no_ImagePositionPatient = [], []

dcm_path_all = []
for idx, k in enumerate(LIDC_IDs):
    #if idx<= 323:continue
    print(f'preprocessing: {idx}, {k}')
    screens = os.listdir(f'{LIDC_PATH}LIDC-IDRI/{k}')
    screens = np.sort(screens)
    
    number_scans = []
    # the whole bunch of scans are located in the folder with most amount of scans
    for idh,h in enumerate(screens):
        new_location = os.listdir(f'{LIDC_PATH}LIDC-IDRI/{k}/{h}/')[0]
        number_scans.append(len(os.listdir(f'{LIDC_PATH}LIDC-IDRI/{k}/{h}/{new_location}')))
        fodler_to_check_idh = number_scans.index(np.max(number_scans))
    scanID = os.listdir(f'{LIDC_PATH}LIDC-IDRI/{k}/{screens[fodler_to_check_idh]}')
    scanID = np.sort(scanID)
    all_scans = os.listdir(f'{LIDC_PATH}LIDC-IDRI/{k}/{screens[fodler_to_check_idh]}/{scanID[0]}')
    all_scans = [i for i in all_scans if 'dcm' in i]
    all_scans = np.sort(all_scans)
    all_scans = [f'{LIDC_PATH}LIDC-IDRI/{k}/{screens[fodler_to_check_idh]}/{scanID[0]}/{i}' for i in all_scans]

    try:
        first_patient = load_scan(all_scans)
    except AttributeError: 
        scans_no_ImagePositionPatient.append(k)
        continue
        
    first_patient_pixels = get_pixels_hu(first_patient)
    pix_resampled, spacing = resample(first_patient_pixels, first_patient, [1,1,1])
    segmented_lungs_fill = segment_lung_mask(pix_resampled, True)
    # Dilate the mask
    selem = ball(1)
    dilated = dilation(segmented_lungs_fill, selem)
    # Apply the mask
    pix_resampled_to_use = pix_resampled*dilated
    
    # The mask
    # put the mask on an array with the same shape as the original volume
    one_segmentation = np.zeros_like(first_patient_pixels)
    
    df_patient = df.loc[df['patientid']==int(k[-4:])]
    pid = k
    for idx_scan in range(pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).count()):
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid)[idx_scan]
        ids = [i.id for i in scan.annotations]
        df_patient_partX = df_patient.loc[df_patient.annotation_id.isin(ids)]
        unique_nodules = np.unique(df_patient_partX['cluster_id'].values)
        for unique_nodule in unique_nodules:
            df_nodule = df_patient_partX.loc[df_patient_partX['cluster_id']==unique_nodule]
        
            scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid)[idx_scan]
            vol = scan.to_volume()
            nods = scan.cluster_annotations()
            anns = nods[unique_nodule]
        
            try:
                cmask,cbbox,masks = consensus(anns, clevel=0.5, pad=[(0,0), (0,0), (0,0)])
            except NameError:
                scans_with_errors.append(pid)
                continue
        
            one_mask = masks[0]
            one_mask = np.swapaxes(one_mask,1,2);one_mask = np.swapaxes(one_mask,0,1)
            one_segmentation[cbbox[2].start:cbbox[2].stop,cbbox[0].start:cbbox[0].stop,cbbox[1].start:cbbox[1].stop]=one_mask
        # Resample
        mask_resampled, _ = resample(one_segmentation, first_patient, [1,1,1])
        
    if not os.path.exists(f'{path_dest}{k}/scans'): os.makedirs(f'{path_dest}{k}/scans')
    if not os.path.exists(f'{path_dest}{k}/masks'): os.makedirs(f'{path_dest}{k}/masks')
    #np.save(f'preprocessed/{k}/preprocessed3d.npy',pix_resampled_to_use)
    for idj,(slice_pix, slice_mask) in enumerate(zip(pix_resampled_to_use, mask_resampled)):
        sparse_matrix = scipy.sparse.csc_matrix(slice_pix)
        sparse_matrix2 = scipy.sparse.csc_matrix(slice_mask)
        scipy.sparse.save_npz(f'{path_dest}{k}/scans/slice_{idj:04d}.npz', sparse_matrix, compressed=True)
        scipy.sparse.save_npz(f'{path_dest}{k}/masks/slice_m_{idj:04d}.npz', sparse_matrix2, compressed=True)
    #if idx==5: break

preprocessing: 0, LIDC-IDRI-0001
Loading dicom files ... This may take a moment.
preprocessing: 1, LIDC-IDRI-0002
Loading dicom files ... This may take a moment.
preprocessing: 2, LIDC-IDRI-0003
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 3, LIDC-IDRI-0004
Loading dicom files ... This may take a moment.
preprocessing: 4, LIDC-IDRI-0005
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 5, LIDC-IDRI-0006
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 6, LIDC-IDRI-0007
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
p

Loading dicom files ... This may take a moment.
preprocessing: 45, LIDC-IDRI-0046
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 46, LIDC-IDRI-0047
Loading dicom files ... This may take a moment.
preprocessing: 47, LIDC-IDRI-0048
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 48, LIDC-IDRI-0049
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.


Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 88, LIDC-IDRI-0089
Loading dicom files ... This may take a moment.
preprocessing: 89, LIDC-IDRI-0090
Loading dicom files ... This may take a moment.
preprocessing: 90, LIDC-IDRI-0091
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 91, LIDC-IDRI-0092
Loading dicom files ... This may take a moment.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Loading dicom files ... This may take a moment.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Loading dicom files ... This may take a moment.
Failed to reduce all groups to

Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 132, LIDC-IDRI-0133
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 133, LIDC-IDRI-0134
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 134, LIDC-IDRI-0135
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 135, LIDC-IDRI-0136
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a mome

/home/om18/anaconda3/envs/misc1/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


...

Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 783, LIDC-IDRI-0786
Loading dicom files ... This may take a moment.
preprocessing: 784, LIDC-IDRI-0787
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 785, LIDC-IDRI-0788
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 786, LIDC-IDRI-0789
Loading dicom files ... This may take a moment.
preprocessing: 787, LIDC-IDRI-0790
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 788, LIDC-IDRI-0791
Loading dicom files ... This may take a moment.
preprocessing: 789, LI

Loading dicom files ... This may take a moment.
preprocessing: 838, LIDC-IDRI-0841
Loading dicom files ... This may take a moment.
preprocessing: 839, LIDC-IDRI-0842
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 840, LIDC-IDRI-0843
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 841, LIDC-IDRI-0844
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 842, LIDC-IDRI-0845
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preproces

Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 877, LIDC-IDRI-0880
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 878, LIDC-IDRI-0881
preprocessing: 879, LIDC-IDRI-0882
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 880, LIDC-IDRI-0883
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 881, LIDC-IDRI-0884
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 882, LIDC-IDRI-0885
preprocessing: 883, LIDC-IDRI-0886
Loading dicom files ... This may ta

Loading dicom files ... This may take a moment.
preprocessing: 938, LIDC-IDRI-0941
Loading dicom files ... This may take a moment.
preprocessing: 939, LIDC-IDRI-0942
Loading dicom files ... This may take a moment.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Loading dicom files ... This may take a moment.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Loading dicom files ... This may take a moment.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
Loading dicom files ... This may take a moment.
Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.
preprocessing: 940, LIDC-IDRI-0943
Loading dicom files ... This may take a moment.
preprocessing: 941, LIDC-IDRI-0944
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preproce

Loading dicom files ... This may take a moment.
preprocessing: 996, LIDC-IDRI-0999
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 997, LIDC-IDRI-1000
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 998, LIDC-IDRI-1001
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 999, LIDC-IDRI-1002
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
Loading dicom files ... This may take a moment.
preprocessing: 1000, LIDC-IDRI-1003
Loading 

In [8]:
11000/17

647.0588235294117